%%latex
\tableofcontents

# Introduction

This notebook describes the analysis of data obtained from the [Pharmacogenetics pipeline](https://github.com/Tuks-ICMM/Pharmacogenetic-Analysis-Pipeline). This includes data handeling and filtering of this data using the python package, [`Pandas`](https://pandas.pydata.org/docs/index.html), and the python graphing library [`seaborn`](https://seaborn.pydata.org/).

> The data used in this notebook notebook was obtained using [Pharmacogenetic Analysis Pipeline](https://github.com/Tuks-ICMM/Pharmacogenetic-Analysis-Pipeline) as part of a project performed under the [Institute for Cellular and Molecular Medicine](https://www.up.ac.za/institute-for-cellular-and-molecular-medicine). The data describes the following genes:

- [CYP2B6](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000197408;r=19:40991282-41018398;t=ENST00000324071)
- [CYP2C9](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000138109;r=10:94938658-94990091;t=ENST00000260682)
- [CYP2C19](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000165841;r=10:94762681-94855547;t=ENST00000371321)
- [CYP2D6](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000100197;r=22:42126499-42130865;t=ENST00000645361)
- [CYP4F2](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000186115;r=19:15878023-15898077;t=ENST00000221700)
- [VKORC1](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000167397;r=16:31090842-31095980;t=ENST00000394975)
- [HLA-A](https://www.ensembl.org/Homo_sapiens/Transcript/Summary?db=core;g=ENSG00000206503;r=6:29941260-29949572;t=ENST00000376809)
- HLA-B

## Objectives

This notebook will focus on the following objectives:

- To identify variants of clinical interest, which fullfill the following criteria:
    - A missingness of less than or equal to 10%
    - Frequency of 2% or higher in any population
    - A CADD Phred score of 10 (CADD-10) or higher
    - A significantly different frequency between the African population and any other comparison population
- To describe the abundance of variants found
    - Variants which survive the above filters
    - Variants which do not survive the above filters

## Notebook Configuration

### Dependancies

We will be making use of the functions shown below to perform basic data import:

In [62]:
from pandas import read_csv, melt, Series
from os.path import join
from seaborn import barplot, set_theme
from matplotlib.pyplot import figure, ylabel, xlabel
from plotly.graph_objects import Figure, Waterfall, Pie
from plotly.subplots import make_subplots
from pathlib import Path
from plotly.io import renderers
from json import load
from os import getcwd
getcwd()

'/mnt/ICMM_SSD_2TB/Graeme/Pharmacogenetic-Analysis-Pipeline'

In [63]:
renderers.default = "notebook_connected+pdf"

This workflow will make use of `input` data files used to perform an analysis using the [Pharmacogenetics Analysis Pipeline](https://github.com/Tuks-ICMM/Pharmacogenetic-Analysis-Pipeline). This will include a `samples.csv` file, which describes all samples included in the analysis, a `locations.csv` file, which describes the gene regions studied, and a `datasets.csv` file which describes the input data used.

In [64]:
with open(join("config", "manifest.json"), 'r') as config:
    config = load(config)

path = join(*config["output"])

In [65]:
# [ASSIGN] the sample metadata used to conduct the analysis to a reference variable
SAMPLES = read_csv(join("input", "samples.csv"))

# [ASSIGN] the genomic location metadata used to conduct the analysis to a reference variable
LOCATIONS = read_csv(join("input", "locations.csv"))

# [ASSIGN] the dataset metadata used to conduct the analysis to a reference variable
DATASETS = read_csv(join("input", "datasets.csv"))

We will also need to compile a few runtime variables for later use.

In [66]:
# [ASSIGN] all unique populations found in our sample annotations
POPULATIONS_TO_COMPARE = SAMPLES["super-population"].unique().tolist()

This analysis will make use of a hirachical index constructed using the chromosome, position, reference allele and alternate allele to present in VCF format, to uniquely identify each record by genomic position and composition. This ensures that we are identifying variants uniquely using biologicaly relevant information.

In [67]:
# [ASSIGN] the columns that form the multiindex
MULTIINDEX = ["CHROM", "POS", "REF", "ALT"]

Lastly, we will be making use of the output of the [Pharmacogenetic Analysis Pipeline](https://github.com/Tuks-ICMM/Pharmacogenetic-Analysis-Pipeline).

In [68]:
DATA = dict()

_total = 0

# [FOR] each gene in our list of gene regions included in this analysis...
for gene in LOCATIONS["location_name"].unique().tolist():

    # [ASSIGN] our data to a key within the DATA dictionary
    DATA[gene] = read_csv(
        join(
            path,
            "consolidated_reports",
            f"super-population_{gene}.csv",
        ),
        sep="\t",
    )
    print(f"Record count | {gene} : {DATA[gene].shape[0]}")
    _total += DATA[gene].shape[0]
print(f"Total record count | {gene} : {_total}")

Record count | CYP2B6 : 1967
Record count | CYP2C9 : 2885
Record count | CYP2C19 : 6081
Record count | CYP2D6 : 435
Record count | CYP4F2 : 983
Record count | VKORC1 : 187
Record count | HLA-A : 614
Record count | HLA-B : 6980
Total record count | HLA-B : 20132


> Throughout this analysis, we will make use of a python dictionary to store the graphs we will be generating across all of our gene regions. For practicality-sake, the graphing operations will be looped and abstracted to generate plots for all genes.

In [69]:
# [ASSIGN] houses all plots we will be making
PLOT = dict()

# [FOR] each gene, create a storage space for any plots generated
# (This should use a hierarchical storage structure of PLOT[gene][graph])
for gene in LOCATIONS["location_name"].unique().tolist():

    # [ASSIGN] an empty dictionary to store all plots made for this gene
    PLOT[gene] = dict()

# [ASSIGN] a generic record to house plots that apply across genes
PLOT[None] = dict()

### Plot configuration

In [70]:
# [CREDIT] for ratio sizes goes to a Bio-render article I found which talks about plot sizes
# https://www.biorender.com/blog/why-layout-and-scale-matters-for-graphs
set_theme(rc={"figure.figsize": (10, 7)}, style="ticks", palette="hls")

In [71]:

for gene in LOCATIONS["location_name"].unique().tolist():
    DATA[gene]["Consequence_type"] = DATA[gene]["Consequence_type"].replace(
        {
            "3_prime_UTR_variant": "3' UTR",
            "5_prime_UTR_variant": "5' UTR",
            "TF_binding_site_variant": "TF Binding Site",
            "downstream_gene_variant": "Downstream",
            "intergenic_variant": "Intergenic",
            "intron_variant": "Intron",
            "missense_variant": "Missense",
            "non_coding_transcript_exon_variant": "Non-Coding Transcript Exon",
            "regulatory_region_variant": "Regulatory Region",
            "splice_acceptor_variant": "Splice Acceptor",
            "splice_donor_5th_base_variant": "Splice Doner 5th Base",
            "splice_donor_region_variant": "Splice Donor Region",
            "splice_polypyrimidine_tract_variant": "Splice Polypyrimidine Tract",
            "splice_region_variant": "Splice Region",
            "stop_gained": "Stop gained",
            "synonymous_variant": "Synonymous",
            "upstream_gene_variant": "Upstream",
        }
    )

## Variant Composition Before Filters

We need to capture the number of variants of each type found before and after for comparison purposes. To accomplish this, we will be using the `make_subplots` function to create a composite plot that we can use to visually illustrate the change in variant counts.


In [72]:
for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]["Variant Filters"] = make_subplots(
        rows=4,
        cols=6,
        specs=[
            [
                {"type": "domain"},
                {"type": "domain"},
                {"type": "domain"},
                {"type": "domain"},
                {"type": "domain"},
                {"type": "domain"},
            ],
            [{"colspan": 6, "rowspan": 3}, None, None, None, None, None],
            [None, None, None, None, None, None],
            [None, None, None, None, None, None],
        ],
    )

Now we can plot the initial round of variant-composition plots.

> We will leave the first row for a waterfall plot illustrating the relative number of variants lost for this gene across each filter applied.

In [73]:
DATA[gene]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,query,Feature_type,Consequence_type,...,EAS_hypothesis_rejection,AFR_P_EAS_corrected,AFR_OR_UAE,AFR_P_UAE,UAE_hypothesis_rejection,AFR_P_UAE_corrected,AFR_OR_SAS,AFR_P_SAS,SAS_hypothesis_rejection,AFR_P_SAS_corrected
0,6,31269498,rs41559915;rs115583167,G,A,191,PASS,6:31269498-31269498:1/A,Transcript,Missense,...,True,3.735617e-32,1.752285,0.028869,False,1.0,1.280289,5.767706e-02,False,1.000000e+00
1,6,31269524,.,A,C,.,PASS,6:31269524-31269524:1/C,Transcript,Splice Region,...,False,1.000000e+00,NaN,1.000000,False,1.0,0.000000,1.903161e-01,False,1.000000e+00
2,6,31269528,rs1433597888,G,A,.,PASS,6:31269528-31269528:1/A,Transcript,Splice Region,...,False,1.000000e+00,NaN,1.000000,False,1.0,NaN,1.000000e+00,False,1.000000e+00
3,6,31269577,rs9264601;rs116571842,G,A,115,PASS,6:31269577-31269577:1/A,Transcript,Intron,...,True,2.357058e-06,2.073394,0.139232,False,1.0,11.880734,5.414052e-11,True,3.779008e-07
4,6,31269580,rs72558162,C,T,.,PASS,6:31269580-31269580:1/T,Transcript,Intron,...,False,1.000000e+00,NaN,1.000000,False,1.0,0.000000,1.457154e-03,False,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6975,6,31357162,rs190427310,G,C,.,PASS,6:31357162-31357162:1/C,Transcript,5' UTR,...,False,1.000000e+00,NaN,1.000000,False,1.0,NaN,1.000000e+00,False,1.000000e+00
6976,6,31357164,rs114038515,C,T,.,PASS,6:31357164-31357164:1/T,Transcript,5' UTR,...,True,1.299079e-35,0.242583,0.005685,False,1.0,0.223735,1.175906e-06,True,8.207824e-03
6977,6,31357170,rs1330011808,A,G,.,PASS,6:31357170-31357170:1/G,Transcript,5' UTR,...,False,1.000000e+00,inf,1.000000,False,1.0,inf,1.000000e+00,False,1.000000e+00
6978,6,31357178,rs34437781,C,T,.,PASS,6:31357178-31357178:1/T,Transcript,5' UTR,...,False,8.322555e-01,inf,0.001945,False,1.0,3.025712,3.770245e-04,False,1.000000e+00


In [74]:

for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]

    _tmp = DATA[gene][["CHROM", "POS", "REF", "ALT", "Consequence_type"]].groupby("Consequence_type").count()
    PLOT[gene]["Variant Filters"].add_trace(Pie(labels=_tmp.index, values=_tmp["CHROM"], scalegroup="Consequence_type", textinfo="percent", textposition="inside", rotation=90), row=1, col=1)
    PLOT[gene]["Variant Filters"].update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

    del _tmp

# Data Cleaning

We are primarily interested in understanding the large-scale impllications of clinicaly relevant mutations in a medical setting in South Africa. Due to this, we will be focusing on variants which are found at a frequency of 2% or more, which will be refered to as <ins>alleles of clinical interest</ins>. 

We will make use of the following criteria to identify these:
- Allelic frequency (2%) in at least one or more population studied
- Variants with a Significantly different frequency in our reference population in at lelast one population comparison


Before we start, lets annotate the `LOCATIONS` object to included a summary of the different types of variants that are kept/removed. This will be usefull when trying to understand why variants have been excluded from analysis.

In [75]:
# [FOR] each gene in our analysis
for gene in LOCATIONS["location_name"].unique().tolist():
    # [ASSIGN] the total number of variants observed
    LOCATIONS.loc[LOCATIONS["location_name"] == gene, "Total Variants"] = DATA[gene].shape[0]
    # [CALCULATE] gene size

## Filter variant missingness

VCF files contain a matrix of indexed genotype records/observations, describing the genomic content at all recorded positions for all samples sequenced and joint-called together. This project makes use of three pre-joint-called VCF files, which does raise some data integrity issues, in the form of unavoidable regions of missing data. Some positions within the genome which were variable in one cohort, may not have been recorded in another. This can artificialy alter allele frequencies, and affect any downstream inference we would otherwise make using observations from the affected regions.

We would like to exclude variants with high misisngness rates. Thankfully, these have been labeled by the [Pharmacogenetic Analysis Pipeline](https://github.com/Tuks-ICMM/Pharmacogenetic-Analysis-Pipeline) already. Variant 'representativeness' was calculated using a function of the total number of observations and twice the number of samples observed:

$$
    \frac{N_{Geno}}{2*N_{s}}*100
$$

All variants with a missingness lower than 91% have been targeted for removal due to poor representation.

In [76]:
def calculate_missingness(row: Series, population: str) -> int:
    if row[f"{population}_tc"] == 0:
        return 100
    else:
        result = (row[f"{population}_ac"] / row[f"{population}_tc"])*100
        return result
    

In [77]:
# [FOR] each gene included in this analysis...
for gene in LOCATIONS["location_name"].unique().tolist():

    _variant_missingness_filters = list()

    # [FOR] each population we have to compare...
    for population in POPULATIONS_TO_COMPARE:
        # [ASSIGN] missingness calculation result
        DATA[gene][f"{population}_missingness"] = DATA[gene].apply(lambda row: calculate_missingness(row, population), axis=1)

        # [TALLY] filter expression for variants with missingness below cutoff in this population
        _variant_missingness_filters.append(f"{population}_missingness <= 10")

    # [FILTER] to variants with missingness below cutoff
    DATA[gene] = DATA[gene].query(" | ".join(_variant_missingness_filters))
    print(f"END | {population} - {DATA[gene].shape[0]}")
    
    # [ASSIGN] the number of alleles identified after our filter to the LOCATIONS DataFrame
    LOCATIONS.loc[LOCATIONS["location_name"] == gene, "Missingness variants remaining"] = DATA[
        gene
    ].shape[0]

END | UAE - 1881
END | UAE - 2837
END | UAE - 5990
END | UAE - 428
END | UAE - 944
END | UAE - 184
END | UAE - 489
END | UAE - 5148


In [78]:

for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]

    _tmp = DATA[gene][["CHROM", "POS", "REF", "ALT", "Consequence_type"]].groupby("Consequence_type").count()
    PLOT[gene]["Variant Filters"].add_trace(Pie(labels=_tmp.index, values=_tmp["CHROM"], scalegroup="Consequence_type", textinfo="percent", textposition="inside", rotation=90), row=1, col=2)
    PLOT[gene]["Variant Filters"].update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

    del _tmp

## Filter frequency cutoff

Since this project focuses primarily on variants which are likely to have an impact on prescriptive outcomes in the Sub-Saharan populations, we would like to remove rare variants as these are not likely to affect a large number of individuals and would require much more specialised interventions.

We can make use of the `query` function to identify and extract all variants found at a 2% frequency threshold in at least one population.

In [79]:
# [FOR] each gene in our list of gene regions included in this analysis...
for gene in LOCATIONS["location_name"].unique().tolist():
    # [COLLECT] a list of population filters that can be passed to the query function
    individual_population_filters = list()

    # [FOR] each unique population in the list of population annotations available for the samples
    for population in POPULATIONS_TO_COMPARE:

        # [TALLY] filters which describe variants above or equal to 2% frequency in at least one population studied
        individual_population_filters.append(f"{population} >= 0.02")

    # Remove in-place the data to keep only alleles found at or above 2% frequency
    DATA[gene] = DATA[gene].query(" | ".join(individual_population_filters))

    # [ASSIGN] the number of alleles identified after our filter to the LOCATIONS DataFrame
    LOCATIONS.loc[LOCATIONS["location_name"] == gene, "Frequency variants remaining"] = DATA[gene].shape[0]

In [80]:

for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]

    _tmp = DATA[gene][["CHROM", "POS", "REF", "ALT", "Consequence_type"]].groupby("Consequence_type").count()
    PLOT[gene]["Variant Filters"].add_trace(Pie(labels=_tmp.index, values=_tmp["CHROM"], scalegroup="Consequence_type", textinfo="percent", textposition="inside", rotation=90), row=1, col=3)
    PLOT[gene]["Variant Filters"].update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

    del _tmp

## Filter by Fishers Exact significance

Thanks to the Fishers-Exact test with Bonferoni-Correction, executed by the [Pharmacogenetic Analysis Pipeline](https://github.com/Tuks-ICMM/Pharmacogenetic-Analysis-Pipeline), we can identify variants which are observed at significantly different rates in our focus population (African populations), compared to the comparison populations in our study.

Here, we will filter for any variants which were found at a significantly different frequency in African samples relative to reference populations.

In [81]:
# [FOR] each gene in our list of genes analysed
for gene in LOCATIONS["location_name"].tolist():
    # [ASSIGN] an empty list to use when tallying up filter expressions for all our populations
    _significant_fishers_exact_filters = list()

    # [FOR] each population in our list of population annotations available for the samples
    for population in POPULATIONS_TO_COMPARE:

        # (Fishers is always a comparison relative to this reference population so there is no column for it,
        # since all other Fishers columns by default use it)
        # [IF] the population is not our reference population
        if population != "AFR":
            # [TALLY] all variants which were marked with significant Fishers-Exact results (w/t Bonferoni Correction)
            _significant_fishers_exact_filters.append(
                f"{population}_hypothesis_rejection == True"
            )

    # [REPLACE] our data with a [QUERY] of all tallied filters describing variants which displayed significantly different frequencies in at least one population
    DATA[gene] = DATA[gene].query(" | ".join(_significant_fishers_exact_filters))

    # [ASSIGN] the number of variants with significant inter-population frequency differences to our LOCATIONS table
    LOCATIONS.loc[
        LOCATIONS["location_name"] == gene, "Fishers Exact variants remaining"
    ] = DATA[gene].shape[0]

In [82]:

for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]

    _tmp = DATA[gene][["CHROM", "POS", "REF", "ALT", "Consequence_type"]].groupby("Consequence_type").count()
    PLOT[gene]["Variant Filters"].add_trace(Pie(labels=_tmp.index, values=_tmp["CHROM"], scalegroup="Consequence_type", textinfo="percent", textposition="inside", rotation=90), row=1, col=4)
    PLOT[gene]["Variant Filters"].update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

    del _tmp

## Filter by Exon

In [83]:
# # [FOR] each gene in our list of genes to analyse...
# for gene in LOCATIONS["location_name"].tolist():
#     DATA[gene] = DATA[gene][DATA[gene]["Exon"].notna()]

#     # [ASSIGN] the number of alleles identified after our filter to the LOCATIONS DataFrame
#     LOCATIONS.loc[LOCATIONS["location_name"] == gene, "Exon variants remaining"] = DATA[gene].shape[0]

## Filter by CADD-Phred label

We will make use of the CADD_PHRED scores provided by the E! Ensembl Data base to identify variants pf predicted deleteriousness. To acomplish this, we will be identifying and labeling variants using _CADD-10_, _CADD-20_ and _CADD-30_ labels, which represent the top 10%, 1% and 0.1% most deleterious mutations possible in the human genome, respectively.

In [84]:
# [FOR] each gene in our list of genes to analyse...
for gene in LOCATIONS["location_name"].tolist():
    DATA[gene]["CADD_Phred_label"] = None
    # print("Before: ", DATA[gene].shape[0])

    # [ASSIGN] a label identifying all variants considered CADD-10 (Top 10% most deleterious SNPs in human genome)
    DATA[gene].loc[DATA[gene]["CADD_PHRED"] >= 10, "CADD_Phred_label"] = "CADD-10"

    # [ASSIGN] a label identifying all variants considered CADD-20 (Top 1% most deleterious SNPs in human genome)
    DATA[gene].loc[DATA[gene]["CADD_PHRED"] >= 20, "CADD_Phred_label"] = "CADD-20"
    
    # [ASSIGN] a label identifying all variants considered CADD-30 (Top 0.1% most deleterious SNPs in human genome)
    DATA[gene].loc[DATA[gene]["CADD_PHRED"] >= 30, "CADD_Phred_label"] = "CADD-30"

    DATA[gene] = DATA[gene][DATA[gene]["CADD_Phred_label"].notna()]
    print("After: ", DATA[gene].shape[0])
    

    # [ASSIGN] the number of alleles identified after our filter to the LOCATIONS DataFrame
    LOCATIONS.loc[LOCATIONS["location_name"] == gene, "CADD Phred variants remaining"] = DATA[gene].shape[0]

After:  5
After:  7
After:  9
After:  14
After:  11
After:  2
After:  71
After:  376


In [85]:

for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]

    _tmp = DATA[gene][["CHROM", "POS", "REF", "ALT", "Consequence_type"]].groupby("Consequence_type").count()
    PLOT[gene]["Variant Filters"].add_trace(Pie(labels=_tmp.index, values=_tmp["CHROM"], scalegroup="Consequence_type", textinfo="percent", textposition="inside", rotation=90), row=1, col=5)
    PLOT[gene]["Variant Filters"].update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

    del _tmp

## Export Filtered Data

In [86]:
# [CHECK] if directory exists, and create if it does not
Path(join(
        path,
        "cleaned")).mkdir(exist_ok=True, parents = True)

We need to keep a reference of all samples which have been identified throughout the above filtering proceses for downstream analysis. Lets export the cleaned data to its own folder for later reference.

In [87]:
_exported_total = 0

# [FOR] gene in the list of genes to analyse
for gene in LOCATIONS["location_name"].unique().tolist():
    print(f"Record count | {gene} : {DATA[gene].shape[0]}")
    _exported_total += DATA[gene].shape[0]
    
    # [EXPORT] to csv the filtered list of samples. This should be used on downstream analyses.
    DATA[gene].to_csv(
        # [COMBINE] function arguments into file-path using OS-apropriate syntax.
        join(
            path,
            "cleaned",
            f"super-population_{gene}.csv.zst",
        ),
        index=False,
    )
print(f"total Record count | {gene} : {_exported_total}")

Record count | CYP2B6 : 5
Record count | CYP2C9 : 7
Record count | CYP2C19 : 9
Record count | CYP2D6 : 14
Record count | CYP4F2 : 11
Record count | VKORC1 : 2
Record count | HLA-A : 71
Record count | HLA-B : 376
total Record count | HLA-B : 495


In [88]:
# [EXPORT] locations.csv file that has now been annotated with abundance information
LOCATIONS.to_csv(
    join(
        path,
        "cleaned",
        f"locations.csv.zst",
    ),
    index=False,
)

# General Plots

## Consequence Type (AFTER)

Now we need to graph the counts of different variants by consequence types.

## Variant abundance

Mutation rates can vary between gene regions, and is not uniform across the genome. Due to this, certain genes, such as the HLA-A and HLA-B genes in this analysis, are known to be hypervariable. Due to its function as a component of the antigen-presenting apparatus of the immune system, HLA-genes are functionaly reliant on high-variation.

To properly visualize various different types and subsets of variant abundance, we will first need to calculate a normalized measure of abundance. To do this, we will have to take into account the length of the gene to accurately identify the rate of variation. This can be done by converting the count into a fraction over the genes total length.

> It is important to mention, that naturally variable abundance rates will still cause changes in abundance score. The abundance ratio used in this lot simply means that spikes in abundance score will not be caused by gene length.

In [89]:
# [CALCULATE] the size of the gene region being studied
LOCATIONS["size"] = LOCATIONS["stop"] - LOCATIONS["start"]

# [CALCULATE] the raw variant abundance ratio before any filters
LOCATIONS["Raw Abundance"] = LOCATIONS["Total Variants"] / LOCATIONS["size"]

# [CALCULATE] variant Missingness-filter abundance ratio
LOCATIONS["Missingness filter"] = (
    LOCATIONS["Missingness variants remaining"] - LOCATIONS["Total Variants"]
) / LOCATIONS["size"]

# [CALCULATE] variant Frequency-filter abundance ratio
LOCATIONS["Frequency filter"] = (
    LOCATIONS["Frequency variants remaining"]
    - LOCATIONS["Missingness variants remaining"]
) / LOCATIONS["size"]

# [CALCULATE] variant Fishers-Exact abundance ratio
LOCATIONS["Fishers-Exact filter"] = (
    LOCATIONS["Fishers Exact variants remaining"]
    - LOCATIONS["Frequency variants remaining"]
) / LOCATIONS["size"]

# [CALCULATE] the CADD Phred abundance of variants with deleterious predictions
LOCATIONS["CADD Phred filter"] = (
    LOCATIONS["Fishers Exact variants remaining"]
    - LOCATIONS["Frequency variants remaining"]
) / LOCATIONS["size"]


# [CALCULATE] the Exon abundance of variants with deleterious predictions
# LOCATIONS["Exon filter"] = (
#     LOCATIONS["Exon variants remaining"]
#     - LOCATIONS["Frequency variants remaining"]
# ) / LOCATIONS["size"]




# Net remaining variants of clinical interest after all filters have been applied.
# This column is set to zero, for use when rendering the waterfall plot, to indicate calculation
# of the net remaining abundance ratios after filtering.
LOCATIONS["NET remaining"] = None


# [SORT] genes by size
LOCATIONS.sort_values(by="size", inplace=True, ascending=False)

In [90]:
LOCATIONS

,location_name,chromosome,start,stop,strand,ld_start,ld_stop,Total Variants,Missingness variants remaining,Frequency variants remaining,Fishers Exact variants remaining,CADD Phred variants remaining,size,Raw Abundance,Missingness filter,Frequency filter,Fishers-Exact filter,CADD Phred filter,NET remaining
2,CYP2C19,10,94762681,94855547,1,92300001,95300000,6081.0,5990.0,646.0,567.0,9.0,92866,0.065481,-0.000980,-0.057545,-0.000851,-0.000851,None
7,HLA-B,6,31269491,31357188,-1,30500001,32100000,6980.0,5148.0,2461.0,2318.0,376.0,87697,0.079592,-0.020890,-0.030640,-0.001631,-0.001631,None
1,CYP2C9,10,94938658,94990091,1,92300001,95300000,2885.0,2837.0,318.0,290.0,7.0,51433,0.056092,-0.000933,-0.048976,-0.000544,-0.000544,None
0,CYP2B6,19,40991282,41018398,1,38200001,42900000,1967.0,1881.0,165.0,134.0,5.0,27116,0.072540,-0.003172,-0.063284,-0.001143,-0.001143,None
4,CYP4F2,19,15878023,15898077,-1,13800001,16100000,983.0,944.0,159.0,149.0,11.0,20054,0.049018,-0.001945,-0.039144,-0.000499,-0.000499,None
5,VKORC1,16,31090854,31095980,-1,28500001,35300000,187.0,184.0,16.0,13.0,2.0,5126,0.036481,-0.000585,-0.032774,-0.000585,-0.000585,None
6,HLA-A,6,29941260,29945884,1,27100001,30500000,614.0,489.0,303.0,288.0,71.0,4624,0.132785,-0.027033,-0.040225,-0.003244,-0.003244,None
3,CYP2D6,22,42126499,42130881,-1,40600001,43800000,435.0,428.0,56.0,48.0,14.0,4382,0.099270,-0.001597,-0.084893,-0.001826,-0.001826,None


Now lets graph the relative variant abundance. For this, I would like to visualise the drop in abundance as we apply each filter to our raw dataset. This will help illustrate where we loose variants, and help explain why.

In [91]:
# PLOT[None]["Variant Filters"] = Figure()
TMP = melt(
    LOCATIONS,
    id_vars="location_name",
    value_vars=[
        "Raw Abundance",
        "Missingness filter",
        "Frequency filter",
        "Fishers-Exact filter",
        "CADD Phred filter",
        "NET remaining"
    ],
    value_name="Abundance",
    var_name="Filter Type",
)

TMP.loc[TMP["Filter Type"] == "NET remaining", "Measure Type"] = "total"
TMP.loc[TMP["Filter Type"] == "Raw Abundance", "Measure Type"] = "absolute"
TMP.loc[(TMP["Filter Type"] != "NET remaining") & (TMP["Filter Type"] != "Raw Abundance"), "Measure Type"] = "relative"

for gene in TMP["location_name"].unique().tolist():
    TMP1 = TMP[TMP["location_name"] == gene]
    labels = LOCATIONS.loc[LOCATIONS["location_name"] == gene][["Total Variants", "Missingness variants remaining", "Frequency variants remaining", "CADD Phred variants remaining", "Fishers Exact variants remaining", "Fishers Exact variants remaining"]]
    PLOT[gene]["Variant Filters"].add_trace(
        Waterfall(
            name=gene,
            measure=TMP1["Measure Type"],
            x=TMP1["Filter Type"],
            y=TMP1["Abundance"],
            # constraintext="inside",
            textposition="outside",
            outsidetextfont={"size": 1, "textcase": "upper"},
            text=labels.values.flatten().tolist(),
            # textinfo="delta",
            # texttemplate="%{label:.0}",
            # textinfo="delta",
            connector={"mode": "spanning", "line": {"width": 1}},
        ),
        row=2,
        col=1
    )
        
    PLOT[gene]["Variant Filters"].update_yaxes(type = 'log')
    PLOT[gene]["Variant Filters"].update_layout(title_text=f"{gene} Variant filters and abundance", 
        waterfallgroupgap = 0.1)
    PLOT[gene]["Variant Filters"].show()
# TODO: Filter TMP to gene-specific and add waterfall trace iteratively

# Export Plots

In [92]:
# [CHECK] if directory exists, and create if it does not
Path(join(path, "Graphs", "01")).mkdir(exist_ok=True, parents = True)

In [93]:
for gene in LOCATIONS["location_name"].unique().tolist():
    for graph in PLOT[gene].keys(): 
        PLOT[gene][graph].write_image(join(path, "Graphs", "01", f"{gene}_{graph}.jpeg"))

for gene in LOCATIONS["location_name"].unique().tolist():
    PLOT[gene]["Variant Filters"].write_image(join(path, "Graphs", "01", "allele_abundance.jpeg"))